In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np
import pandas as pd
import datatable as dt
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices

In [ ]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
import random
from sklearn import metrics



FOLDS = 5
SEED = 42
NANFILL = -9999
#gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
#splits = list(gkf.split(y, groups=train['date'].values))



In [ ]:
def concat_respx_pred(X,models):
    y_multi_pre = np.stack([ models[f'{respx}'].predict_proba(X)[:,1] for respx in models if respx != 'lr']).T
    return y_multi_pre
def model_predict_slow(X,models):
    y_multi_pre = concat_respx_pred(X,models)
    y = models['lr'].predict(y_multi_pre)
    #y = y > np.mean(y)
    #print(np.mean(y))
    return y.astype(int)

def model_predict_fast(X,models):
    
    y = models['resp'].predict(X)
    #y = y > np.mean(y)
    #print(np.mean(y))
    return y.astype(int)

In [ ]:
import pickle
with open('../input/models/lgb_model_ensemble_0208_01.pkl', 'rb') as f:
    models = pickle.load(f)


In [ ]:
import janestreet
from tqdm import tqdm
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

model_predict = model_predict_slow
features = [f'feature_{x}' for x in range(130)]
for (test_df, sample_prediction_df) in tqdm(iter_test):
    if test_df['weight'].values[0] < 0.001:
        y_preds = 0
    else:
        X_test = test_df.loc[:, features].fillna(NANFILL).values
        y_preds = model_predict(X_test,models)
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)